In [1]:
# standard
import numpy as np
import pandas as pd
import polars as pl
from sklearn.model_selection import train_test_split
import dask.dataframe as dd

# Import necessary libraries
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.init as init
import matplotlib.pyplot as plt

from ignite.engine import create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import FastaiLRFinder

# generator1 = torch.Generator().manual_seed(42)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Supported Device: ", DEVICE)

# Decide random state
torch.manual_seed(42)
torch.set_default_dtype(torch.float32) #this is a bug, if using ADAM optimizer need to explicit in all operations. Can't just set default to float64
DTYPE = torch.float32

Supported Device:  cuda


In [2]:
# Define ResNet architecture without Batch Normalization
class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ResidualBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(in_features, out_features, dtype=DTYPE),
            nn.ReLU(inplace=True).double(),
            nn.Linear(out_features, out_features, dtype=DTYPE),
            nn.ReLU(inplace=True).double(),
            nn.Linear(out_features, out_features, dtype=DTYPE),
        )
        if (in_features != out_features):
            self.fc = nn.Linear(in_features, out_features, dtype=DTYPE)
        else:
            self.fc = nn.Identity(dtype=DTYPE)
    
    def forward(self, x):
        return self.fc(x) + self.layer(x)

class ResNet(nn.Module):
    def __init__(self, in_features, out_features, hidden_features, num_blocks):
        super(ResNet, self).__init__()
        
        layers = [ResidualBlock(in_features, hidden_features)]
        layers += [ResidualBlock(hidden_features, hidden_features) for _ in range(num_blocks-2)]
        layers.append(ResidualBlock(hidden_features, out_features))

        self.network = nn.Sequential(*layers)
        self.fc = nn.Linear(out_features, out_features, dtype=DTYPE)
    
    def forward(self, x):
        out = self.network(x)
        return self.fc(out)
# He initialization function
def weights_init(m):
    if isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight, nonlinearity='relu')  # He initialization
        if m.bias is not None:
            init.zeros_(m.bias)
            
def r2_score_custom(y_pred:torch.Tensor, y_true:torch.Tensor) -> float:
    
    ss_res = torch.sum((y_true - y_pred) ** 2)
    ss_tot = torch.sum((y_true - torch.mean(y_true)) ** 2)
    
    r2 = 1 - ss_res / ss_tot
    
    return r2.item()

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)


In [3]:
MIN_STD = 1e-8
SCHEDULER_PATIENCE = 3
SCHEDULER_FACTOR = 10**(-0.5)
PATIENCE = 6
PRINT_FREQ = 50

In [4]:
%%time
# Import each file
# temporarily set n_rows for saving time
n_rows = 500_000
folder = 'leap-atmospheric-physics-ai-climsim'
df_train = pl.read_csv('../input/'+folder+'/train.csv', n_rows=n_rows).to_pandas()
df_sub = pl.read_csv('../input/'+folder+'/sample_submission.csv', n_rows=2).to_pandas()

CPU times: user 49.7 s, sys: 15.3 s, total: 1min 5s
Wall time: 37.2 s


In [5]:
torch.manual_seed(42)
weights = df_sub.iloc[0, 1:].values.astype(np.float64)
weights = torch.tensor(weights, dtype=DTYPE)

# Prepare data for training
x_train = torch.tensor(1 * df_train.iloc[:, 1:557].values, dtype=DTYPE)
y_train = torch.tensor(1 * df_train.iloc[:, 557:].values, dtype=DTYPE)
y_train = y_train * weights

# Normalization of training data
mx = x_train.mean(axis=0)
sx = np.maximum(x_train.std(axis=0), MIN_STD)
x_train = (x_train - mx.reshape(1,-1)) / sx.reshape(1,-1)

my = y_train.mean(axis=0)
sy = np.maximum(np.sqrt((y_train*y_train).mean(axis=0)), MIN_STD)
y_train = (y_train - my.reshape(1,-1)) / sy.reshape(1,-1)


# Create a dataset and shuffle it
dataset = TensorDataset(x_train, y_train)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

y_t = y_train[train_dataset.indices, :].to(DEVICE, dtype = DTYPE)
y_v = y_train[val_dataset.indices, :].to(DEVICE, dtype = DTYPE)

SS_res_train = torch.sum((y_t - y_t.mean())**2)
SS_res_val = torch.sum((y_v - y_v.mean())**2)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1_000, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1_000, shuffle=False,)

In [6]:
torch.manual_seed(42)

# Model parameters
input_dim = x_train.shape[1]
output_dim = y_train.shape[1]
hidden_dim = np.int64(np.mean([input_dim, output_dim]).round()) #hidden layers is the mean of the number of inputs and outputs
num_res_blocks = 7

# Create ResNet model
model = ResNet(input_dim, output_dim, hidden_dim, num_res_blocks)
model = model.to(DEVICE, dtype = DTYPE)

# Apply the He weight initialization (does not work well)
# model.apply(weights_init)

num_epochs = 50

# Training settings
criterion = nn.MSELoss(reduction = "mean").to(DEVICE, dtype = DTYPE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=SCHEDULER_FACTOR, patience=SCHEDULER_PATIENCE)

# lr_finder.apply_suggested_lr(optimizer)

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_in, batch_out in train_loader:
        
        batch_in = batch_in.to(DEVICE, dtype = DTYPE)
        batch_out = batch_out.to(DEVICE, dtype = DTYPE)

        optimizer.zero_grad()
        predictions = model(batch_in)
        
        loss = criterion(predictions, batch_out)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_in, batch_out in val_loader:
            batch_in = batch_in.to(DEVICE, dtype = DTYPE)
            batch_out = batch_out.to(DEVICE, dtype = DTYPE)

            predictions = model(batch_in)
            loss = criterion(predictions, batch_out)
            val_loss += loss.item()
#     scheduler.step()
    if (epoch + 1) % 5 == 0:
        Rsq_train = (train_loss * train_loader.batch_size * output_dim) / SS_res_train; Rsq_train = 1 - Rsq_train
        Rsq_test = (val_loss * val_loader.batch_size * output_dim) / SS_res_val; Rsq_test = 1 - Rsq_test
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')
        print(f'Epoch [{epoch+1}/{num_epochs}], Train $R^2$: {Rsq_train:.4f}, Val $R^2$: {Rsq_test:.4f}')

Epoch [5/50], Train Loss: 0.4273, Val Loss: 0.4747
Epoch [5/50], Train $R^2$: 0.4479, Val $R^2$: 0.4205
Epoch [10/50], Train Loss: 0.5042, Val Loss: 0.4785
Epoch [10/50], Train $R^2$: 0.3484, Val $R^2$: 0.4158
Epoch [15/50], Train Loss: 0.3697, Val Loss: 0.4600
Epoch [15/50], Train $R^2$: 0.5223, Val $R^2$: 0.4385
Epoch [20/50], Train Loss: 0.3515, Val Loss: 0.4523
Epoch [20/50], Train $R^2$: 0.5458, Val $R^2$: 0.4478
Epoch [25/50], Train Loss: 0.3757, Val Loss: 0.4575
Epoch [25/50], Train $R^2$: 0.5145, Val $R^2$: 0.4414
Epoch [30/50], Train Loss: 0.3217, Val Loss: 0.4840
Epoch [30/50], Train $R^2$: 0.5842, Val $R^2$: 0.4091
Epoch [35/50], Train Loss: 0.3169, Val Loss: 0.4785
Epoch [35/50], Train $R^2$: 0.5905, Val $R^2$: 0.4158
Epoch [40/50], Train Loss: 0.3015, Val Loss: 0.6187
Epoch [40/50], Train $R^2$: 0.6104, Val $R^2$: 0.2446
Epoch [45/50], Train Loss: 0.2859, Val Loss: 0.4706
Epoch [45/50], Train $R^2$: 0.6306, Val $R^2$: 0.4255
Epoch [50/50], Train Loss: 0.2859, Val Loss: 0.4

In [7]:
torch.save(model.state_dict(), "best_model.pth")

In [8]:
model = ResNet(input_dim, output_dim, hidden_dim, num_res_blocks)
model = model.to(DEVICE)
model.load_state_dict(torch.load("best_model.pth", map_location=DEVICE))

<All keys matched successfully>

In [9]:
import gc 
gc.collect()
torch.cuda.empty_cache()

In [10]:
df_test = pl.read_csv('../input/'+folder+'/test.csv')
df_test = df_test.to_pandas()
df_test = df_test.set_index("sample_id")

df_subm = pl.read_csv('../input/'+folder+'/sample_submission.csv')
df_subm = df_subm.to_pandas()
df_subm = df_subm.set_index("sample_id")

In [11]:
# Prepare data for training
x_test = torch.tensor(1 * df_test.to_numpy()).to(DEVICE, dtype=DTYPE)
x_test = (x_test - mx.to(DEVICE, dtype=DTYPE).reshape(1,-1)) / sx.to(DEVICE, dtype=DTYPE).reshape(1,-1)

model.eval()
with torch.no_grad():
        y_est = model(x_test)

In [12]:
TARGET_NAMES = ['ptend_t_0', 'ptend_t_1', 'ptend_t_2', 'ptend_t_3', 'ptend_t_4', 'ptend_t_5', 'ptend_t_6', 'ptend_t_7', 'ptend_t_8', 'ptend_t_9', 'ptend_t_10', 'ptend_t_11', 'ptend_t_12', 'ptend_t_13', 'ptend_t_14', 'ptend_t_15', 'ptend_t_16', 'ptend_t_17', 'ptend_t_18', 'ptend_t_19', 'ptend_t_20', 'ptend_t_21', 'ptend_t_22', 'ptend_t_23', 'ptend_t_24', 'ptend_t_25', 'ptend_t_26', 'ptend_t_27', 'ptend_t_28', 'ptend_t_29', 'ptend_t_30', 'ptend_t_31', 'ptend_t_32', 'ptend_t_33', 'ptend_t_34', 'ptend_t_35', 'ptend_t_36', 'ptend_t_37', 'ptend_t_38', 'ptend_t_39', 'ptend_t_40', 'ptend_t_41', 'ptend_t_42', 'ptend_t_43', 'ptend_t_44', 'ptend_t_45', 'ptend_t_46', 'ptend_t_47', 'ptend_t_48', 'ptend_t_49', 'ptend_t_50', 'ptend_t_51', 'ptend_t_52', 'ptend_t_53', 'ptend_t_54', 'ptend_t_55', 'ptend_t_56', 'ptend_t_57', 'ptend_t_58', 'ptend_t_59', 'ptend_q0001_0', 'ptend_q0001_1', 'ptend_q0001_2', 'ptend_q0001_3', 'ptend_q0001_4', 'ptend_q0001_5', 'ptend_q0001_6', 'ptend_q0001_7', 'ptend_q0001_8', 'ptend_q0001_9', 'ptend_q0001_10', 'ptend_q0001_11', 'ptend_q0001_12', 'ptend_q0001_13', 'ptend_q0001_14', 'ptend_q0001_15', 'ptend_q0001_16', 'ptend_q0001_17', 'ptend_q0001_18', 'ptend_q0001_19', 'ptend_q0001_20', 'ptend_q0001_21', 'ptend_q0001_22', 'ptend_q0001_23', 'ptend_q0001_24', 'ptend_q0001_25', 'ptend_q0001_26', 'ptend_q0001_27', 'ptend_q0001_28', 'ptend_q0001_29', 'ptend_q0001_30', 'ptend_q0001_31', 'ptend_q0001_32', 'ptend_q0001_33', 'ptend_q0001_34', 'ptend_q0001_35', 'ptend_q0001_36', 'ptend_q0001_37', 'ptend_q0001_38', 'ptend_q0001_39', 'ptend_q0001_40', 'ptend_q0001_41', 'ptend_q0001_42', 'ptend_q0001_43', 'ptend_q0001_44', 'ptend_q0001_45', 'ptend_q0001_46', 'ptend_q0001_47', 'ptend_q0001_48', 'ptend_q0001_49', 'ptend_q0001_50', 'ptend_q0001_51', 'ptend_q0001_52', 'ptend_q0001_53', 'ptend_q0001_54', 'ptend_q0001_55', 'ptend_q0001_56', 'ptend_q0001_57', 'ptend_q0001_58', 'ptend_q0001_59', 'ptend_q0002_0', 'ptend_q0002_1', 'ptend_q0002_2', 'ptend_q0002_3', 'ptend_q0002_4', 'ptend_q0002_5', 'ptend_q0002_6', 'ptend_q0002_7', 'ptend_q0002_8', 'ptend_q0002_9', 'ptend_q0002_10', 'ptend_q0002_11', 'ptend_q0002_12', 'ptend_q0002_13', 'ptend_q0002_14', 'ptend_q0002_15', 'ptend_q0002_16', 'ptend_q0002_17', 'ptend_q0002_18', 'ptend_q0002_19', 'ptend_q0002_20', 'ptend_q0002_21', 'ptend_q0002_22', 'ptend_q0002_23', 'ptend_q0002_24', 'ptend_q0002_25', 'ptend_q0002_26', 'ptend_q0002_27', 'ptend_q0002_28', 'ptend_q0002_29', 'ptend_q0002_30', 'ptend_q0002_31', 'ptend_q0002_32', 'ptend_q0002_33', 'ptend_q0002_34', 'ptend_q0002_35', 'ptend_q0002_36', 'ptend_q0002_37', 'ptend_q0002_38', 'ptend_q0002_39', 'ptend_q0002_40', 'ptend_q0002_41', 'ptend_q0002_42', 'ptend_q0002_43', 'ptend_q0002_44', 'ptend_q0002_45', 'ptend_q0002_46', 'ptend_q0002_47', 'ptend_q0002_48', 'ptend_q0002_49', 'ptend_q0002_50', 'ptend_q0002_51', 'ptend_q0002_52', 'ptend_q0002_53', 'ptend_q0002_54', 'ptend_q0002_55', 'ptend_q0002_56', 'ptend_q0002_57', 'ptend_q0002_58', 'ptend_q0002_59', 'ptend_q0003_0', 'ptend_q0003_1', 'ptend_q0003_2', 'ptend_q0003_3', 'ptend_q0003_4', 'ptend_q0003_5', 'ptend_q0003_6', 'ptend_q0003_7', 'ptend_q0003_8', 'ptend_q0003_9', 'ptend_q0003_10', 'ptend_q0003_11', 'ptend_q0003_12', 'ptend_q0003_13', 'ptend_q0003_14', 'ptend_q0003_15', 'ptend_q0003_16', 'ptend_q0003_17', 'ptend_q0003_18', 'ptend_q0003_19', 'ptend_q0003_20', 'ptend_q0003_21', 'ptend_q0003_22', 'ptend_q0003_23', 'ptend_q0003_24', 'ptend_q0003_25', 'ptend_q0003_26', 'ptend_q0003_27', 'ptend_q0003_28', 'ptend_q0003_29', 'ptend_q0003_30', 'ptend_q0003_31', 'ptend_q0003_32', 'ptend_q0003_33', 'ptend_q0003_34', 'ptend_q0003_35', 'ptend_q0003_36', 'ptend_q0003_37', 'ptend_q0003_38', 'ptend_q0003_39', 'ptend_q0003_40', 'ptend_q0003_41', 'ptend_q0003_42', 'ptend_q0003_43', 'ptend_q0003_44', 'ptend_q0003_45', 'ptend_q0003_46', 'ptend_q0003_47', 'ptend_q0003_48', 'ptend_q0003_49', 'ptend_q0003_50', 'ptend_q0003_51', 'ptend_q0003_52', 'ptend_q0003_53', 'ptend_q0003_54', 'ptend_q0003_55', 'ptend_q0003_56', 'ptend_q0003_57', 'ptend_q0003_58', 'ptend_q0003_59', 'ptend_u_0', 'ptend_u_1', 'ptend_u_2', 'ptend_u_3', 'ptend_u_4', 'ptend_u_5', 'ptend_u_6', 'ptend_u_7', 'ptend_u_8', 'ptend_u_9', 'ptend_u_10', 'ptend_u_11', 'ptend_u_12', 'ptend_u_13', 'ptend_u_14', 'ptend_u_15', 'ptend_u_16', 'ptend_u_17', 'ptend_u_18', 'ptend_u_19', 'ptend_u_20', 'ptend_u_21', 'ptend_u_22', 'ptend_u_23', 'ptend_u_24', 'ptend_u_25', 'ptend_u_26', 'ptend_u_27', 'ptend_u_28', 'ptend_u_29', 'ptend_u_30', 'ptend_u_31', 'ptend_u_32', 'ptend_u_33', 'ptend_u_34', 'ptend_u_35', 'ptend_u_36', 'ptend_u_37', 'ptend_u_38', 'ptend_u_39', 'ptend_u_40', 'ptend_u_41', 'ptend_u_42', 'ptend_u_43', 'ptend_u_44', 'ptend_u_45', 'ptend_u_46', 'ptend_u_47', 'ptend_u_48', 'ptend_u_49', 'ptend_u_50', 'ptend_u_51', 'ptend_u_52', 'ptend_u_53', 'ptend_u_54', 'ptend_u_55', 'ptend_u_56', 'ptend_u_57', 'ptend_u_58', 'ptend_u_59', 'ptend_v_0', 'ptend_v_1', 'ptend_v_2', 'ptend_v_3', 'ptend_v_4', 'ptend_v_5', 'ptend_v_6', 'ptend_v_7', 'ptend_v_8', 'ptend_v_9', 'ptend_v_10', 'ptend_v_11', 'ptend_v_12', 'ptend_v_13', 'ptend_v_14', 'ptend_v_15', 'ptend_v_16', 'ptend_v_17', 'ptend_v_18', 'ptend_v_19', 'ptend_v_20', 'ptend_v_21', 'ptend_v_22', 'ptend_v_23', 'ptend_v_24', 'ptend_v_25', 'ptend_v_26', 'ptend_v_27', 'ptend_v_28', 'ptend_v_29', 'ptend_v_30', 'ptend_v_31', 'ptend_v_32', 'ptend_v_33', 'ptend_v_34', 'ptend_v_35', 'ptend_v_36', 'ptend_v_37', 'ptend_v_38', 'ptend_v_39', 'ptend_v_40', 'ptend_v_41', 'ptend_v_42', 'ptend_v_43', 'ptend_v_44', 'ptend_v_45', 'ptend_v_46', 'ptend_v_47', 'ptend_v_48', 'ptend_v_49', 'ptend_v_50', 'ptend_v_51', 'ptend_v_52', 'ptend_v_53', 'ptend_v_54', 'ptend_v_55', 'ptend_v_56', 'ptend_v_57', 'ptend_v_58', 'ptend_v_59', 'cam_out_NETSW', 'cam_out_FLWDS', 'cam_out_PRECSC', 'cam_out_PRECC', 'cam_out_SOLS', 'cam_out_SOLL', 'cam_out_SOLSD', 'cam_out_SOLLD']

In [13]:
%%time 

prediction = (y_est * sy.to(DEVICE, dtype=DTYPE).reshape(1,-1)) + my.to(DEVICE, dtype=DTYPE).reshape(1,-1)

prediction = prediction.cpu().detach().numpy()
print("Saving Test File")
df_subm_final = df_subm.astype(np.float64) * 1
df_subm_final.loc[:, :] = prediction
df_subm_final = df_subm_final.reset_index()
df_subm_final = df_subm_final[["sample_id"] + TARGET_NAMES]
df_subm_final = pl.from_pandas(df_subm_final)
df_subm_final.write_csv("submission.csv")
df_subm_final

Saving Test File
CPU times: user 31.2 s, sys: 11.5 s, total: 42.7 s
Wall time: 28.2 s


sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_169651""",0.121571,-0.493538,-0.083397,-0.299344,-0.616368,-0.859764,-0.921237,-0.908393,-1.004644,-0.989049,-0.885063,-0.794898,-0.789876,-0.604074,-0.257513,0.089419,0.022484,0.26962,0.125336,-0.060211,0.103092,-0.28003,-0.47849,-0.61908,-0.71699,-0.866627,-1.012016,-0.901568,-0.909278,-0.838012,-0.788432,-0.701559,-0.628489,-0.567237,-0.570446,-0.432552,…,-0.087128,-0.084063,-0.186856,-0.133083,-0.024662,0.049324,-0.055132,0.030734,0.178703,0.220072,0.170313,0.057763,-0.0694,-0.025766,0.094257,-0.029559,-0.067677,-0.01339,0.118927,0.152087,0.09077,0.087334,-0.104625,-0.219307,-0.427933,-0.206692,0.129918,0.282764,0.295666,0.005056,4.911753,0.055448,-0.089787,0.000189,0.090915,0.195587,0.269291
"""test_524862""",0.144425,-0.374275,-0.142555,-0.245702,-0.649414,-0.872517,-0.872439,-0.821513,-0.921117,-0.957905,-0.900094,-0.815395,-0.857144,-0.770775,-0.475102,-0.116852,-0.269393,0.191344,0.054855,-0.167499,-0.017105,-0.351726,-0.343926,-0.414926,-0.45212,-0.471876,-0.666639,-0.667187,-0.714477,-0.677714,-0.67082,-0.626127,-0.570778,-0.523806,-0.544749,-0.471153,…,-0.421759,-0.289868,-0.070149,0.221203,0.31819,0.281163,0.008129,0.04753,0.199146,0.208815,0.141451,0.01645,-0.071009,-0.000312,0.070846,-0.11899,-0.144341,-0.07479,0.058668,0.028752,-0.027213,0.007125,-0.091738,-0.157638,-0.263367,-0.082796,0.094031,0.251855,0.43265,0.013998,4.731476,0.012559,0.060134,-0.013782,0.070702,0.158087,0.250763
"""test_634129""",0.034685,-0.979847,-0.361393,-0.369399,-0.643838,-0.954246,-0.992321,-0.927706,-0.888605,-0.836025,-0.759955,-0.750483,-0.829596,-0.690155,-0.344751,0.148764,0.191434,0.484561,0.0856,-0.423054,0.305648,-0.075834,-0.509585,-0.679575,-0.830861,-1.018635,-1.05901,-0.760274,-0.840035,-0.900288,-0.923677,-0.878317,-0.814862,-0.653375,-0.694742,-0.610921,…,0.135022,0.119046,-0.057644,0.010471,0.092298,0.061475,-0.143132,-0.009314,0.161412,0.170543,0.051311,-0.055051,-0.16005,-0.100986,0.039179,-0.16602,-0.243188,-0.175802,-0.083968,-0.016128,0.059292,0.104703,0.044858,0.071813,-0.127641,-0.143441,0.115547,0.342091,0.179289,0.070306,5.299164,-0.050974,-0.050818,0.093235,0.140507,0.247853,0.406688
"""test_403572""",0.041548,-0.858429,-0.337164,-0.297285,-0.542609,-0.845865,-0.927849,-0.87988,-0.851609,-0.783587,-0.712319,-0.679666,-0.777433,-0.720369,-0.545828,-0.176126,-0.222048,0.513194,0.178578,-0.369111,0.196623,-0.328883,-0.529524,-0.709634,-0.701098,-0.771716,-0.999405,-0.915578,-0.922318,-0.946914,-1.018447,-0.995175,-0.874624,-0.664878,-0.60899,-0.483311,…,-0.790158,-0.769188,-0.361485,0.138744,0.434993,0.38477,-0.017797,0.01763,0.17562,0.061629,-0.162557,-0.29382,-0.237584,-0.060741,0.026885,-0.237984,-0.325212,-0.3277,-0.173546,-0.109202,-0.006871,0.092854,0.061904,0.168294,0.220281,0.44127,0.288891,0.032564,-0.106579,0.149063,5.41166,-0.008102,0.213669,0.114002,0.146753,0.204655,0.328838
"""test_4845